In [2]:
import json
import re
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import TfidfVectorizer 
import pandas as pd
import csv
import os
import pickle
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [3]:
# Helper functions

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    try:
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)    
    except FileNotFoundError as e:
        return False;

In [7]:
# Turn initial dataset to csv from json files

import csv
with open('../data/dataset/dataset_filtered.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["title", "header", "recitals", "main_body", "attachments", "concepts"])

    directory = os.fsencode('../data/dataset/train_filtered/')
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".json"): 
            concepts = []
            title = ""
            header = ""
            recitals = ""
            main_body = ""
            attachments = ""

            with open('../data/dataset/train_filtered/' + str(filename), encoding='utf-8') as json_file:
                data = json.load(json_file)
                concepts = data["concepts"]
                title = data["title"]
                header = data["header"]
                recitals = data["recitals"]
                main_body = '\n'.join(data["main_body"])
                attachments = data["attachments"]
                writer.writerow([title, header, recitals, main_body, attachments, concepts])

In [4]:
# FIND No. OF OCCURANCES OF EACH CONCEPT AND FILTER OUT ONLY ONES THAT APPEAR IN MORE THAT 10 DOCS.
# Note: the dictionary with this data (key: CONCEPT_ID, value: NUMBER_OF_OCCURANCES) is saved in file 'dict_concept_num'.
# Note 2: Filtered dict contains 1289 concepts.

dict = {}
directory = os.fsencode('../data/dataset/train/')
counter = 0  
print(counter)
dict = load_obj('helper_objects/dict_concept_num')
if dict:
    print('True')
else:
    dict = {}
    for file in os.listdir(directory):
        counter = counter + 1
        filename = os.fsdecode(file)
        if filename.endswith(".json"): 
            concepts = []
            title = ""
            header = ""
            recitals = ""
            main_body = ""
            attachments = ""

            with open('../data/dataset/train/' + str(filename), encoding='utf-8') as json_file:
                data = json.load(json_file)
                concepts = data["concepts"]
                for concept in concepts:
                    if concept in dict:
                      dict[concept] = dict[concept] + 1
                    else:
                      dict[concept] = 1
    #save_obj(dict,'helper_objects/dict_concept_num')
filtered_dict = {k:v for (k,v) in dict.items() if v > 10}   
sum_values = 0
for (k,v) in dict.items():
    sum_values = sum_values + v
print(sum_values)

print(len(filtered_dict))
print(len(dict))
save_obj(filtered_dict, "helper_objects/dict_concept_filtered")

0
True
122882
451
3612


In [6]:
# REMOVE PREVIOUSLY REMOVED CONCEPTS FROM DOCUMENTS and CREATE NEW DATASET WITH DOCUMENTS WITH LEFTOVER CONCEPTS. 
# Note: this removes concepts that have exclusively concepts that are removed in the prevous step.
# Note 2: this removes only ~ 300-400 docs.

directory = os.fsencode('../data/dataset/train/')
dict_rez = {}
counter = 0
for file in os.listdir(directory):
    counter = counter + 1
    filename = os.fsdecode(file)
#     if counter == 7:
#         break
    if filename.endswith(".json"): 
        concepts = []
        title = ""
        header = ""
        recitals = ""
        main_body = ""
        attachments = ""

        with open('../data/dataset/train/' + str(filename), encoding='utf-8') as json_file:
            data = json.load(json_file)
            concepts = data["concepts"]
            rez_concepts = []
            for concept in concepts:
                if concept in filtered_dict:
                  rez_concepts.append(concept)
                  dict_rez[concept] = dict_rez[concept] + 1 if concept in dict_rez else 1
            if len(rez_concepts) > 0:
                data["concepts"] = rez_concepts
                with open('../data/dataset/train_filtered/' + str(filename), 'w') as outfile:
                    json.dump(data, outfile)
print(len(dict_rez))  

451
